# 미션 2: 카카오 대화 요약


## 🎯 학습 목표

이 미션에서는 카카오톡 대화 내역을 분석하고 요약하는 AI 서비스를 개발합니다.

### 주요 학습 내용
- 대화형 텍스트 데이터 처리
- 멀티턴 대화 요약 기법
- 화자별 대화 분리 및 분석

## 📝 실습 내용

### 구현 기능
- 카카오톡 대화 데이터 파싱
- 화자별 대화 분리
- 대화 내용 요약
- 주요 키워드 추출

### 사용 기술
- Python
- OpenAI API / LangChain
- Jupyter Notebook


### LLM 모델
- gpt-3.5-turb

## 라이브러리 로드

In [ ]:
# 데이터 파싱
import glob
import json
import os
import pickle

# AI
import time
import openai
import anthropic
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv()

ANTHROPIC_API_KEY = os.environ['ANTHROPIC_API_KEY']
GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

## 카카오톡 대화 데이터 파싱 & 화자별 대화 분리

### Train data

In [3]:
conversations = []
paths = glob.glob('./res/297.SNS_데이터_고도화/01-1.정식개방데이터/Training/02.라벨링데이터/TL/*json')
target_count = 20
count = 0
for path in paths:
    with open(path, 'r') as f:
        conv_dict = json.load(f)
        # 2 명 선택
        if conv_dict['header']['dialogueInfo']['numberOfParticipants'] == 2:
            # 30 글자 이상
            if conv_dict['header']['dialogueInfo']['numberOfUtterances'] > 30:
                conv_list = []
                for d in conv_dict['body']:
                    conv_list.append(d['participantID'] + ': ' + d['utterance'])
                if conv_list[0] == conv_list[1]:
                    continue
                conv_text = '\n'.join(conv_list)
                conversations.append(conv_text)
                if count == target_count:
                    break


In [4]:
with open('./res/train_data.pickle', 'wb') as f:
    pickle.dump(conversations, f)

In [5]:
conversations[-2].split('\n')

['P01: 매직해도 머릿결 안상함?',
 'P01: 얇아지거나 전혀 영향없나 요샌...',
 'P02: 상하지 얇아지진 않즈 키키',
 'P02: 얇아지는건 두피건강이니까',
 'P02: 근데 매직 정도는 마니 안 상하게 충분히 가능',
 'P01: **샘한테 얼마주고 햇우',
 'P01: 나도 머리 이번엔 좀 길러봐야겠엉',
 'P02: 5마넌 줬어',
 'P02: 3마넌 달라길래.ㅌㅌ',
 'P01: 키키 야귀엽누 키키',
 'P01: 다른데가서 니 머리 길이 매직이면 10은 거뜬히 나올 텐데',
 'P02: 애매한게 내가 샘머리 짤라줬거든',
 'P02: 난 뿌리매직만 하공',
 'P02: 키키 긍서 2마넌 달라는 거야',
 'P02: 글서 쌤 너무 심한거 아니에요? 이러니까 3달래',
 'P01: 키키',
 'P02: 글서 5쥼 키키',
 'P01: 나 장염땜에 어제 하루종일 비워냇는데도 계속먹어서 그런지 살이 1도 안빠졌네...',
 'P02: 내가 양꼬치도 사줫거든 키키',
 'P02: 장염을 이기는 식이네',
 'P01: 장염 걸리면 수분이라도 빠져서 무게빠지는게 국룰인데...',
 'P02: 그러니깐 키키',
 'P02: 근데 나도 그렇더라',
 'P02: 먹으니까 이기더라',
 'P01: 키키 아침에 일어났는데 몸이 가벼운느낌이 아니라 부은 느낌이엇어 키키',
 'P02: 키키 웃겨',
 'P02: 어젯밤에 많이 먹었니',
 'P01: 어제밤에 생라면을 좀 먹긴했지... 키키 짯나...?',
 'P01: 짠과자가 땡기더라구 키키',
 'P01: 다이어트 언제하니',
 'P02: 키키 다이어트 오늘부터 하자...']

### 데이터 생성

In [6]:
conversations = []

paths = glob.glob('./res/297.SNS_데이터_고도화/01-1.정식개방데이터/Validation/02.라벨링데이터/VL/*json')
target_count = 20
count = 0
for path in paths:
    with open(path, 'r') as f:
        conv_dict = json.load(f)
        if conv_dict['header']['dialogueInfo']['numberOfParticipants'] == 2:
            if conv_dict['header']['dialogueInfo']['numberOfUtterances'] > 30:
                conv_list = []
                for d in conv_dict['body']:
                    conv_list.append(d['participantID'] + ': ' + d['utterance'])
                if conv_list[0] == conv_list[1]:
                    print('Repeated Conversations')
                    continue
                conv_text = '\n'.join(conv_list)
                conversations.append(conv_text)
                if count == target_count:
                    break
count = 0
target_count = 30
for path in paths:
    with open(path, 'r') as f:
        conv_dict = json.load(f)
        if conv_dict['header']['dialogueInfo']['numberOfParticipants'] > 2:
            if conv_dict['header']['dialogueInfo']['numberOfUtterances'] > 30:
                conv_list = []
                for d in conv_dict['body']:
                    conv_list.append(d['participantID'] + ': ' + d['utterance'])
                if conv_list[0] == conv_list[1]:           
                    continue
                conv_text = '\n'.join(conv_list)
                conversations.append(conv_text)
                if count == target_count:
                    break

In [7]:
prompt = f"""You are given a conversation among three users below. Imagine what the users discussed prior to the conversation.
Please write the previous content, strictly following the tone and format of given conversation, with at least 3000 characters.

{conversations[-2]}
"""

client = anthropic.Anthropic(
    api_key=os.environ['ANTHROPIC_API_KEY']
)

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=4096,
    temperature=0.0,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

/tmp/ipykernel_3577494/2244774912.py:11: DeprecationWarning: The model 'claude-3-5-sonnet-20240620' is deprecated and will reach end-of-life on October 22, 2025.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  message = client.messages.create(


In [8]:
lst = message.content[0].text.split('\n\n')[2:]

In [9]:
lst

[]

In [10]:
with open('./res/conv_long.pickle', 'rb') as f:
    conv_long = pickle.load(f)


### Test data

In [11]:
conversations = []

paths = glob.glob('./res/297.SNS_데이터_고도화/01-1.정식개방데이터/Validation/02.라벨링데이터/VL/*json')
target_count = 20
count = 0

for path in paths:
    with open(path, 'r') as f:
        conv_dict = json.load(f)
        if conv_dict['header']['dialogueInfo']['numberOfParticipants'] == 2:
            if conv_dict['header']['dialogueInfo']['numberOfUtterances'] > 30:
                conv_list = []
                for d in conv_dict['body']:
                    conv_list.append(d['participantID'] + ': ' + d['utterance'])
                if conv_list[0] == conv_list[1]:
                    print('Repeated Conversations')
                    continue
                conv_text = '\n'.join(conv_list)
                conversations.append(conv_text)
                count += 1
                if count == target_count:
                    break

In [12]:
target_count = 30
count = 0

for path in paths:
    with open(path, 'r') as f:
        conv_dict = json.load(f)
        if conv_dict['header']['dialogueInfo']['numberOfParticipants'] > 2:
            if conv_dict['header']['dialogueInfo']['numberOfUtterances'] > 30:
                conv_list = []
                for d in conv_dict['body']:
                    conv_list.append(d['participantID'] + ': ' + d['utterance'])
                if conv_list[0] == conv_list[1]:
                    continue
                conv_text = '\n'.join(conv_list)
                conversations.append(conv_text)
                count += 1
                if count == target_count:
                    break

In [13]:
with open('./res/conv_long.pickle', 'rb') as f:
    conv_long = pickle.load(f)


In [14]:
conversations[-2] = '\n'.join(conv_long) + '\n' + conversations[-2]

In [15]:
conversations[-2]

'P02: 야 요즘 뭐하고 지내?\nP01: 나? 그냥 평소처럼 일하고 집에서 쉬고 그러지\nP01: 너는?\nP03: 나도 비슷해\nP03: 요즘 코로나 때문에 밖에 나가기도 좀 그렇고\nP02: 맞아 나도 그래\nP02: 그래서 집에서 할 거 찾다가 넷플릭스 보고 있어\nP01: 아 맞다 넷플릭스\nP01: 나도 요즘 넷플릭스 많이 보는데\nP03: 나는 유튜브를 더 많이 보는 것 같아\nP03: 넷플릭스는 가끔씩?\nP02: 유튜브도 좋지\nP02: 근데 넷플릭스는 화질이 좋잖아\nP01: 그러게\nP01: 나는 넷플릭스랑 유튜브 둘 다 많이 봐\nP03: 그래? 넷플릭스에서 뭐 재밌는 거 봤어?\nP02: 나는 요즘 범죄 다큐멘터리 보고 있어\nP02: 넷플릭스에 그런 거 많더라고\nP01: 오 범죄 다큐멘터리?\nP01: 나는 그런 건 잘 못 봐\nP01: 무서워서...\nP03: 나도 그런 건 별로야\nP03: 차라리 코미디나 로맨스 같은 거 보는 게 나아\nP02: 키키 그래?\nP02: 난 오히려 그런 게 재밌던데\nP01: 각자 취향이 다른가 보다\nP01: 나는 요즘 한국 드라마를 많이 봐\nP03: 한국 드라마?\nP03: 뭐 봤는데?\nP01: 응 킹덤이랑 스위트홈 봤어\nP01: 둘 다 넷플릭스 오리지널이야\nP02: 아 나도 킹덤 봤어!\nP02: 좀비물인데 재밌더라고\nP03: 난 그런 건 또 무서워서 못 봐\nP03: 그냥 가벼운 로맨스 코미디가 좋아\nP01: 키키 그래?\nP01: 로맨스 코미디 추천해줘\nP03: 음... 나는 최근에 청춘기록 봤어\nP03: 박보검이랑 박소담 나오는 거\nP02: 아 그거 들어본 것 같아\nP02: 근데 난 안 봤어\nP01: 나도 안 봤는데 어때?\nP03: 나는 재밌게 봤어\nP03: 배우들도 예쁘고 잘생겼고\nP02: 키키 역시 외모 보는구나\nP01: 그러게 키키\nP01: 나도 배우 외모 보고 드라마 고르는 편이야\nP03: 아니야 내용도 좋았어!\nP03: 청춘들의 고민이랑 성장을 

In [16]:
with open('./res/eval_data.pickle', 'wb') as f:
    pickle.dump(conversations, f)

In [17]:
def get_eval_data():
    with open('./res/eval_data.pickle', 'rb') as f:
        eval_data = pickle.load(f)

    return eval_data

## 대화 내용 요약

In [18]:
MAX_LEN = 3000

In [19]:
def shorten_conv(conversation):
    shortened_len = len(conversation)
    lst = conversation.split('\n')
    for i, l in enumerate(lst):
        utterance_len = len(l)
        shortened_len -= utterance_len
        if shortened_len <= MAX_LEN:
            break

    lst_shortened = lst[i+1:]
    conv_shortened = '\n'.join(lst_shortened)
    return conv_shortened


def summarize(conversation, prompt, temperature=0.0, model='gpt-4.1-nano'):
    if len(conversation) > MAX_LEN:
        conversation = shorten_conv(conversation)

    prompt = prompt + '\n\n' + conversation

    if 'gpt' in model:
        client = OpenAI(api_key=OPENAI_API_KEY)
        completion = client.chat.completions.create(
            model=model,
            messages=[{'role': 'user', 'content': prompt}],
            temperature=temperature
        )

        return completion.choices[0].message.content
    elif 'gemini' in model:
        genai.configure(api_key=GOOGLE_API_KEY)
        client = genai.GenerativeModel(model)
        response = client.generate_content(
            contents=prompt,
            safety_settings={
                HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE
            }
        )
        time.sleep(1)

        return response.text
    elif 'claude' in model:
        client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
        message = client.messages.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=1024
        )

        return message.content[0].text


def get_train_data():
    with open('./res/train_data.pickle', 'rb') as f:
        train_data = pickle.load(f)

    return train_data


def get_summary_prompt():
    conv_train = get_train_data()[18]

    prompt = f"""당신은 요약 전문가입니다. 사용자 대화들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 대화를 요약할 때는 다음 단계를 따라주세요:

1. 대화 참여자 파악: 대화에 참여하는 사람들의 수와 관계를 파악합니다.
2. 주제 식별: 대화의 주요 주제와 부차적인 주제들을 식별합니다.
3. 핵심 내용 추출: 각 주제에 대한 중요한 정보나 의견을 추출합니다.
4. 감정과 태도 분석: 대화 참여자들의 감정이나 태도를 파악합니다.
5. 맥락 이해: 대화의 전반적인 맥락과 배경을 이해합니다.
6. 특이사항 기록: 대화 중 특별히 눈에 띄는 점이나 중요한 사건을 기록합니다.
7. 요약문 작성: 위의 단계에서 얻은 정보를 바탕으로 간결하고 명확한 요약문을 작성합니다.
각 단계를 수행한 후, 최종적으로 전체 대화를 200자 내외로 요약해주세요.

아래는 예시 대화와 예시 요약 과정 및 결과 입니다.

예시 대화:
{conv_train}

예시 요약 과정
1. "우리 대학교 졸업 여행 간 거 기억나?"라는 언급과 전반적으로 친밀한 대화 톤을 사용하고 있는 것을 보았을 떄 두 사용자는 오랜 친구 사이로 보입니다.
대화의 시작 부분에서 "코로나가 좀 잠잠해지면 해외여행 중에 가고 싶은 곳 있어?"라고 묻고 있는 것을 보았을 때 코로나 이후 가고 싶은 해외 여행지에 대해 논의하고 있습니다.
따라서 다음과 같이 요약 할 수 있습니다:
최소 대학 생활부터 함께 한 매우 친밀한 사이의 두 사용자가 코로나가 잠잠해졌을 때 방문하고 싶은 해외 여행지에 대해 일상적이고 가벼운 톤으로 대화하고 있습니다.

2. 대화 중 호주, 일본, 하와이, 괌, 베트남 다낭, 스위스, 유럽들이 언급하고 있습니다.
남편의 첫 직장 워크샵, 대학교 졸업 여행, 호주 워킹홀리데이 등의 경험을 이야기하면서 과거 여행 경험을 공유하며 추억을 회상하고 있습니다.
따라서 다음과 같이 요약 할 수 있습니다:
여행지로는 하와이, 괌, 스위스, 호주, 베트남 다낭 등을 언급하며 남편과의 연락 관련 다툼이나 졸업여행 관련 추억을 회상합니다.

3. 소매치기, 여권 분실, 인도에서의 여성 여행자 위험 등을 언급하며 해외 여행의 위험성에 대해 우려를 표현하고 있습니다.
"해외 여행 가면 가이드 안 끼고 가면 영어 실력 엄청 좋은 사람이랑 가는 거 아닐 땐 소통 문제도 좀 곤란할 때가 있는 거 같아"라는 언급과 "왜 영어 공부를 열심히 안 했을까... 후회"라는 표현이 있는 것을 보았을 때 언어 장벽의 어려움을 인식하고 영어 실력 향상에 대한 욕구를 표현합니다.
따라서 다음과 같이 요약 할 수 있습니다:
또한 여행 중 발생하는 위험에 대한 우려도 표하고 있으며, 해외여행 시 언어 장벽의 어려움을 인식하고 영어 실력을 향상시키고 싶다는 마음을 가볍게 표현합니다.

예시 요약 결과
최소 대학 생활부터 함께 한 매우 친밀한 사이의 두 사용자가 코로나가 잠잠해졌을 때 방문하고 싶은 해외 여행지에 대해 일상적이고 가벼운 톤으로 대화하고 있습니다.
여행지로는 하와이, 괌, 스위스, 호주, 베트남 다낭 등을 언급하며 남편과의 연락 관련 다툼이나 졸업여행 관련 추억을 회상합니다.
또한 여행 중 발생하는 위험에 대한 우려도 표하고 있으며, 해외여행 시 언어 장벽의 어려움을 인식하고 영어 실력을 향상시키고 싶다는 마음을 가볍게 표현합니다.
    
아래 사용자 대화에 대해 3문장 내로 요약해주세요:"""
    return prompt

In [20]:

def get_eval_data():
    with open('./res/eval_data.pickle', 'rb') as f:
        eval_data = pickle.load(f)

    return eval_data

In [ ]:
model = 'gpt-4.1-mini'

summary_result_dict = {model: []}
prompt = get_summary_prompt()
for i in range(5):
    conversation = get_eval_data()[i]
    summary = summarize(
        conversation=conversation,
        prompt=prompt,
        model=model
    )
    summary_result_dict[model].append({'index' : i, 'summary' : summary, "conversation" : conversation})

  0%|          | 0/5 [00:00<?, ?it/s]

In [25]:
summary_result_dict

{'gpt-4.1-mini': [{'index': 0,
   'summary': '1. 대화 참여자 파악: 두 명(P01, P02)이 친근한 사이로 서울에서 갈 만한 디저트 카페와 전시회에 대해 이야기하고 있습니다.  \n2. 주제 식별: 서울에서 디저트 카페 방문과 대림 전시회 관람 계획이 주요 주제입니다.  \n3. 핵심 내용 추출: P02는 달달한 빵이나 조각케익을 먹고 싶어 하고, P01은 너무 단 것은 싫어하며 마카롱과 케이크는 별로라고 합니다. 두 사람은 서울에서 디저트 카페에 가기로 하고, P01은 대림 전시회도 가고 싶어 합니다.  \n4. 감정과 태도 분석: 두 사람 모두 가벼운 기대감과 긍정적인 태도를 보이며, 함께 외출할 계획에 즐거워합니다.  \n5. 맥락 이해: 평소 디저트 카페 방문이 드문 P02가 오랜만에 가고 싶어 하고, P01은 서울에서의 문화 활동도 함께 제안합니다.  \n6. 특이사항 기록: 대림 전시회가 무료라는 점에 두 사람이 호감을 보입니다.  \n\n요약:  \n두 명이 서울에서 디저트 카페와 대림 전시회 방문을 계획하며, P02는 달콤한 빵과 조각케익을 선호하고 P01은 너무 단 디저트는 피하려 합니다. 두 사람은 함께 외출할 기대감에 긍정적이며, 대림 전시회의 무료 관람에 관심을 보입니다.',
   'conversation': 'P01: 서울에 갈곳 없나\nP02: 누난요새 디저트카페 이런곳가고싶다\nP02: 안가본지오래돼서\nP01: 어디요?\nP02: 그냥 빵집이런곳\nP01: 디저트 근데 누나 마카롱같은거 싫어하잖아\nP02: 달달한거 먹고싶어\nP02: 근데 빵은먹잖아\nP01: 단거 투성이들 별로 싫어하지않나\nP02: 아 그렇게 단거말고\nP01: 디저트하면 마카롱밖에 생각안남\nP02: 그냥 뭐 그냥 빵들\nP01: 아하...\nP02: 마카롱을 그냥 없애봐\nP01: 나는 너무 달면 별로야 키키\nP02: 빵도 달음\nP01: 난 케익도 싫어\nP02: 나도 난케익그냥 조각케익정도?\nP01: 난 솔직

## 주요 키워드 추출

In [28]:

def get_keyword_prompt():
    conv_train = get_train_data()[18]
    prompt = f"""당신은 키워드 요약 전문가입니다. 사용자 대화들이 주어졌을 때 키워드 추출하는 것이 당신의 목표입니다. 키워드를 추출할 때는 다음 단계를 따라주세요.

1. 대화 참여자 파악: 대화에 참여하는 사람들의 수와 관계를 파악합니다.
2. 주제 식별: 대화의 주요 주제와 부차적인 주제들을 식별합니다.
3. 핵심 내용 추출: 각 주제에 대한 중요한 정보나 의견을 추출합니다.
4. 감정과 태도 분석: 대화 참여자들의 감정이나 태도를 파악합니다.
5. 맥락 이해: 대화의 전반적인 맥락과 배경을 이해합니다.
6. 특이사항 기록: 대화 중 특별히 눈에 띄는 점이나 중요한 사건을 기록합니다.
7. 키워드 추출 : 요약문과 대화 기록을 바탕으로 위의 단계에서 얻은 정보를 바탕으로 간결하고 명확한 키워드들을 작성합니다.
각 단계를 수행한 후, 최종적으로 전체 대화를 10 개 내외의 키워드로 요약해주세요.
8. 키워드만 추출해주세요. 나머지는 출력은 절대 하지 마세요.
"""
    return prompt

In [30]:
def extract_keyword(conversation, summary, prompt, temperature=0.0, model='gpt-4.1-nano'):
    if len(conversation) > MAX_LEN:
        conversation = shorten_conv(conversation)

    prompt = prompt + '\n\n' + conversation + '\n\n' + summary

    if 'gpt' in model:
        client = OpenAI(api_key=OPENAI_API_KEY)
        completion = client.chat.completions.create(
            model=model,
            messages=[{'role': 'user', 'content': prompt}],
            temperature=temperature
        )

        return completion.choices[0].message.content
    elif 'gemini' in model:
        genai.configure(api_key=GOOGLE_API_KEY)
        client = genai.GenerativeModel(model)
        response = client.generate_content(
            contents=prompt,
            safety_settings={
                HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE
            }
        )
        time.sleep(1)

        return response.text
    elif 'claude' in model:
        client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
        message = client.messages.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=1024
        )

        return message.content[0].text

In [ ]:
prompt = get_keyword_prompt()
for i in range(5):
    conversation = get_eval_data()[i]
    keyword = extract_keyword(
        conversation=conversation,
        summary=summary,
        prompt=prompt,
        model=model
    )
    summary_result_dict[model][i]['keyword'] = keyword

  0%|          | 0/5 [00:00<?, ?it/s]

In [32]:
summary_result_dict

{'gpt-4.1-mini': [{'index': 0,
   'summary': '1. 대화 참여자 파악: 두 명(P01, P02)이 친근한 사이로 서울에서 갈 만한 디저트 카페와 전시회에 대해 이야기하고 있습니다.  \n2. 주제 식별: 서울에서 디저트 카페 방문과 대림 전시회 관람 계획이 주요 주제입니다.  \n3. 핵심 내용 추출: P02는 달달한 빵이나 조각케익을 먹고 싶어 하고, P01은 너무 단 것은 싫어하며 마카롱과 케이크는 별로라고 합니다. 두 사람은 서울에서 디저트 카페에 가기로 하고, P01은 대림 전시회도 가고 싶어 합니다.  \n4. 감정과 태도 분석: 두 사람 모두 가벼운 기대감과 긍정적인 태도를 보이며, 함께 외출할 계획에 즐거워합니다.  \n5. 맥락 이해: 평소 디저트 카페 방문이 드문 P02가 오랜만에 가고 싶어 하고, P01은 서울에서의 문화 활동도 함께 제안합니다.  \n6. 특이사항 기록: 대림 전시회가 무료라는 점에 두 사람이 호감을 보입니다.  \n\n요약:  \n두 명이 서울에서 디저트 카페와 대림 전시회 방문을 계획하며, P02는 달콤한 빵과 조각케익을 선호하고 P01은 너무 단 디저트는 피하려 합니다. 두 사람은 함께 외출할 기대감에 긍정적이며, 대림 전시회의 무료 관람에 관심을 보입니다.',
   'conversation': 'P01: 서울에 갈곳 없나\nP02: 누난요새 디저트카페 이런곳가고싶다\nP02: 안가본지오래돼서\nP01: 어디요?\nP02: 그냥 빵집이런곳\nP01: 디저트 근데 누나 마카롱같은거 싫어하잖아\nP02: 달달한거 먹고싶어\nP02: 근데 빵은먹잖아\nP01: 단거 투성이들 별로 싫어하지않나\nP02: 아 그렇게 단거말고\nP01: 디저트하면 마카롱밖에 생각안남\nP02: 그냥 뭐 그냥 빵들\nP01: 아하...\nP02: 마카롱을 그냥 없애봐\nP01: 나는 너무 달면 별로야 키키\nP02: 빵도 달음\nP01: 난 케익도 싫어\nP02: 나도 난케익그냥 조각케익정도?\nP01: 난 솔직

## 결과

In [38]:
print(summary_result_dict[model][0]['conversation'])

P01: 서울에 갈곳 없나
P02: 누난요새 디저트카페 이런곳가고싶다
P02: 안가본지오래돼서
P01: 어디요?
P02: 그냥 빵집이런곳
P01: 디저트 근데 누나 마카롱같은거 싫어하잖아
P02: 달달한거 먹고싶어
P02: 근데 빵은먹잖아
P01: 단거 투성이들 별로 싫어하지않나
P02: 아 그렇게 단거말고
P01: 디저트하면 마카롱밖에 생각안남
P02: 그냥 뭐 그냥 빵들
P01: 아하...
P02: 마카롱을 그냥 없애봐
P01: 나는 너무 달면 별로야 키키
P02: 빵도 달음
P01: 난 케익도 싫어
P02: 나도 난케익그냥 조각케익정도?
P01: 난 솔직히 그냥 조각케익 정도
P02: 응응  조각케익이  먹기조아
P01: 큰거 하나사면 별로임
P02: 그건 이제 사람 많을 때
P01: 근데 조각케익은 주제넘게 비싸
P02: ... 그거였군
P01: 서울에 디저트 카페 가보자
P02: 오 진짜 가자 ㅜㅜ
P01: 대림에 나는 전시회 가고싶어
P02: *** 안가려해  ?
P02: 거기머하는데
P01: 예전에 가보고싶엇어
P02: 뭔데
P01: 저기 공짜라서 좋아했음 키키
P02: 키키 공짜좋지


In [37]:
print(summary_result_dict[model][0]['summary'])

1. 대화 참여자 파악: 두 명(P01, P02)이 친근한 사이로 서울에서 갈 만한 디저트 카페와 전시회에 대해 이야기하고 있습니다.  
2. 주제 식별: 서울에서 디저트 카페 방문과 대림 전시회 관람 계획이 주요 주제입니다.  
3. 핵심 내용 추출: P02는 달달한 빵이나 조각케익을 먹고 싶어 하고, P01은 너무 단 것은 싫어하며 마카롱과 케이크는 별로라고 합니다. 두 사람은 서울에서 디저트 카페에 가기로 하고, P01은 대림 전시회도 가고 싶어 합니다.  
4. 감정과 태도 분석: 두 사람 모두 가벼운 기대감과 긍정적인 태도를 보이며, 함께 외출할 계획에 즐거워합니다.  
5. 맥락 이해: 평소 디저트 카페 방문이 드문 P02가 오랜만에 가고 싶어 하고, P01은 서울에서의 문화 활동도 함께 제안합니다.  
6. 특이사항 기록: 대림 전시회가 무료라는 점에 두 사람이 호감을 보입니다.  

요약:  
두 명이 서울에서 디저트 카페와 대림 전시회 방문을 계획하며, P02는 달콤한 빵과 조각케익을 선호하고 P01은 너무 단 디저트는 피하려 합니다. 두 사람은 함께 외출할 기대감에 긍정적이며, 대림 전시회의 무료 관람에 관심을 보입니다.


In [39]:
print(summary_result_dict[model][0]['keyword'])

서울, 디저트카페, 빵집, 마카롱, 단맛, 조각케익, 가격, 전시회, 대림, 무료
